# 7차시: 텐서플로우 2.x 활용 자연어 생성

## AI 맛보기 7주차: 2020. 08. 18. 20:00 ~ 22:00 (120분)
1. 도구 불러오기 및 버전 확인
1. 학습 데이터 다운로드
1. 학습 데이터 살펴보기
1. 데이터 전처리
1. 모델 생성
1. 자연어 생성 이해하기
1. 모델 학습
1. 자연어 생성

#### 참고자료
- [파이썬 3 표준 문서](https://docs.python.org/3/index.html)
- [자연어 생성](https://www.tensorflow.org/tutorials/text/text_generation)

### 1. 도구 불러오기 및 버전 확인

In [ ]:
import os

import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [ ]:
print(f'Tensorflow 버전을 확인합니다: {tf.__version__}')

### 2. 학습 데이터 다운로드

In [ ]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt',
                                       'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [ ]:
text = open(path_to_file, 'r').read()

print (f'Length of text: {len(text)} characters')

### 3. 학습 데이터 살펴보기

In [ ]:
print(text[:250])

In [ ]:
vocab = sorted(set(text))
print (f'{len(vocab)} unique characters')

### 4. 데이터 전처리

In [ ]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [ ]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print(f'  {repr(char):4s}: {char2idx[char]:3d},')
print('  ...\n}')

In [ ]:
print (f'{repr(text[:13])} ---- characters mapped to int ---- > {text_as_int[:13]}')

In [ ]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

In [ ]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

In [ ]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset_map = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in  dataset_map.take(1):
    print(f'Input data: {repr("".join(idx2char[input_example.numpy()]))}')
    print(f'Target data: {repr("".join(idx2char[target_example.numpy()]))}')

In [ ]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print(f'Step {i:4d}')
    print(f'  input: {input_idx} ({idx2char[input_idx]:s})')
    print(f'  expected output: {target_idx} ({repr(idx2char[target_idx]):s})')

### 5. 모델 생성

In [ ]:
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = dataset_map.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

print(dataset)

In [ ]:
vocab_size = len(vocab)

embedding_dim = 256

rnn_units = 1024

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
                    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                              batch_input_shape=[batch_size, None]),
                    tf.keras.layers.GRU(rnn_units,
                                        return_sequences=True,
                                        stateful=True,
                                        recurrent_initializer='glorot_uniform'),
                    tf.keras.layers.Dense(vocab_size)
                    ])
    return model

In [ ]:
model = build_model(vocab_size=len(vocab),
                    embedding_dim=embedding_dim,
                    rnn_units=rnn_units,
                    batch_size=BATCH_SIZE)

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(f'{example_batch_predictions.shape}, # (batch_size, sequence_length, vocab_size)')

In [ ]:
model.summary()

### 6. 자연어 생성 이해하기

In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()
print(sampled_indices)

In [ ]:
print(f'Input: \n{repr("".join(idx2char[input_example_batch[0]]))}')
print()
print(f'Next Char Predictions: \n{repr("".join(idx2char[sampled_indices ]))}')

In [ ]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print(f'Prediction shape: {example_batch_predictions.shape}, # (batch_size, sequence_length, vocab_size)')
print(f'scalar_loss:      {example_batch_loss.numpy().mean()}')

### 7. 모델 학습

In [ ]:
model.compile(optimizer='adam', loss=loss)

In [ ]:
checkpoint_dir = '/tmp/training_checkpoints'

checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt_{epoch}')

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                                         save_weights_only=True)

In [ ]:
epochs=10*5

history = model.fit(dataset, 
                    epochs=epochs, 
                    callbacks=[checkpoint_callback])

### 8. 자연어 생성

In [ ]:
print(tf.train.latest_checkpoint(checkpoint_dir))

predict_model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

predict_model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

predict_model.build(tf.TensorShape([1, None]))

predict_model.summary()

In [ ]:
def generate_text(model, start_string, num_generate=1000):
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = list()

    temperature = 1.0

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)

        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [ ]:
print(generate_text(predict_model, start_string='ROMEO: '))